In [2]:
import pandas as pd
from pathlib import Path
import re

current_dir = Path.cwd()
project_root = current_dir.parent
csv_path = project_root / "data/raw" / "customer_dump.csv"
df = pd.read_csv(csv_path, sep=';', index_col='client_id')

In [3]:
df.head()

,full_name,email,phone,registration_date,account_status
client_id,,,,,
1000,Allison Hill,donaldgarcia@example.net,(219)560-0133,2011-04-12,1
1001,Jennifer Cole,lisa02@example.net,354-723-5116,2018-11-09,Active
1002,Renee Blair,dudleynicholas@example.net,395-593-1034,14/12/2019,N
1003,Jesse Flowers,donaldgarcia@example.net,5554341928,1996-08-25,1
1004,Timothy Watts,gabriellecameron@example.org,5059641395,22/10/1992,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5000 entries, 1000 to 5999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   full_name          5000 non-null   object
 1   email              5000 non-null   object
 2   phone              4775 non-null   object
 3   registration_date  4939 non-null   object
 4   account_status     5000 non-null   object
dtypes: object(5)
memory usage: 234.4+ KB


In [5]:
df.isnull().sum()

full_name              0
email                  0
phone                225
registration_date     61
account_status         0
dtype: int64

In [6]:
df.index.is_unique

True

In [52]:
df.duplicated().sum()

np.int64(0)

In [7]:
df['email'].duplicated().sum()


np.int64(531)

In [8]:
df['phone'].duplicated().sum()

np.int64(224)

In [9]:
df['account_status'].value_counts()

account_status
1         2486
N          991
0          775
Active     479
Y          269
Name: count, dtype: int64

In [10]:
def normalize_format(phone) -> str:
    if pd.isna(phone):
        return "NaN"
    phone_str = str(phone)
    return re.sub(r"\d", "X", phone_str)

In [11]:
phone_formats = df['phone'].apply(normalize_format)
phone_formats.value_counts()

phone
XXXXXXXXXX       1618
XXX-XXX-XXXX     1588
(XXX)XXX-XXXX    1569
NaN               225
Name: count, dtype: int64

In [12]:
def normalize_to_iso(date_str: str) -> str:
    if pd.isna(date_str):
        return date_str
    
    date_str = str(date_str).strip()
    
    if '/' in date_str:
        day, month, year = date_str.split('/')
        return f'{year}-{month}-{day}'
    elif '.' in date_str:
        day, month, year = date_str.split('.')
        return f'{year}-{month}-{day}'
    return date_str

In [13]:
df['dates_iso'] = df['registration_date'].apply(normalize_to_iso)

In [14]:
df['dates_parsed'] = pd.to_datetime(
    df['dates_iso'],
    errors='coerce'
)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5000 entries, 1000 to 5999
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   full_name          5000 non-null   object        
 1   email              5000 non-null   object        
 2   phone              4775 non-null   object        
 3   registration_date  4939 non-null   object        
 4   account_status     5000 non-null   object        
 5   dates_iso          4939 non-null   object        
 6   dates_parsed       4939 non-null   datetime64[ns]
dtypes: datetime64[ns](1), object(6)
memory usage: 441.5+ KB


In [23]:
names_split = df['full_name'].str.split(' ', expand=True)
names_split.tail()

,0,1,2,3
client_id,,,,
5995,Patrick,Gonzalez,None,None
5996,Andrea,Burns,None,None
5997,James,Smith,Jr.,None
5998,Alan,Casey,None,None
5999,Patricia,Bennett,None,None


In [24]:
names_split.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5000 entries, 1000 to 5999
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       5000 non-null   object
 1   1       5000 non-null   object
 2   2       215 non-null    object
 3   3       23 non-null     object
dtypes: object(4)
memory usage: 324.4+ KB


In [36]:
id_2 = []
for idx, value in names_split[2].items():
    if value:
        id_2.append(idx)

In [37]:
id_3 = []
for idx, value in names_split[3].items():
    if value:
        id_3.append(idx)

In [51]:
all_problematic_records = set(id_2 + id_3)
print(len(all_problematic_records), all_problematic_records)

215 {3586, 2565, 1543, 2567, 4615, 4627, 3092, 1557, 2582, 2583, 1048, 1049, 2074, 3611, 1564, 4119, 4632, 5144, 1063, 5673, 5164, 4142, 4655, 5680, 3636, 1591, 3128, 2617, 2618, 5689, 5182, 1599, 4162, 1093, 3659, 3662, 4176, 5204, 1621, 3162, 4189, 5213, 2147, 3171, 4708, 2664, 1129, 3583, 4717, 3182, 5744, 4721, 5746, 1139, 2676, 3189, 3190, 3701, 4212, 1145, 2682, 1150, 3199, 1153, 1156, 4229, 3207, 4743, 3209, 5770, 2188, 3220, 3223, 4759, 4250, 1692, 3744, 1702, 2215, 5802, 2733, 4272, 1713, 3762, 5808, 1716, 3764, 1209, 2233, 1212, 5310, 3776, 5824, 1218, 3267, 4291, 4293, 4802, 1223, 4808, 5320, 3276, 1741, 3278, 1743, 5324, 5841, 3802, 3806, 3807, 3809, 4322, 5865, 1261, 3821, 4845, 2289, 3313, 5874, 1788, 1277, 1796, 4357, 5140, 1293, 5901, 3856, 3857, 3861, 2326, 4376, 3867, 5916, 3357, 2846, 5406, 5654, 5415, 1834, 2348, 3886, 5426, 2357, 1846, 2873, 5950, 4416, 2885, 4422, 4424, 2378, 4938, 1870, 3408, 3920, 5968, 4435, 2901, 5974, 4953, 1882, 4444, 1374, 2398, 2399, 5984,